In [13]:
!pip install torch torchvision torchaudio


In [14]:
import torch
import torch.nn as nn
import numpy as np
import itertools
import torch.optim as optim
import os

from typing import Literal
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold


print("CUDA available:", torch.cuda.is_available())
print("Device:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")



#BASE_RESULTS_DIR = "lstm_results"
BASE_RESULTS_DIR = "/kaggle/working/lstm_results"

WITHIN_DIR = os.path.join(BASE_RESULTS_DIR, "within_subject")
CROSS_DIR = os.path.join(BASE_RESULTS_DIR, "cross_subject")

os.makedirs(WITHIN_DIR, exist_ok=True)
os.makedirs(CROSS_DIR, exist_ok=True)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

#DATA_PATH = "data/data.npz"
DATA_PATH = "/kaggle/input/data-npz/data.npz"
CHANNELS = ["EEG.AF3", "EEG.T7", "EEG.Pz", "EEG.T8", "EEG.AF4"]
CHANNEL_TO_IDX = { c: idx for idx, c in enumerate(CHANNELS) }

CUDA available: True
Device: Tesla T4
Using device: cuda


In [15]:
def eval_on_loader_bce(model: nn.Module, loader: DataLoader, device: str):
    model.eval()
    loss_fn = nn.BCEWithLogitsLoss()
    losses, correct, total = [], 0, 0

    for xb, yb in loader:
        xb = xb.to(device)
        yb = yb.to(device)

        logits = model(xb)
        loss = loss_fn(logits, yb)
        losses.append(loss.item())

        probs = torch.sigmoid(logits)
        preds = (probs >= 0.5).float()
        correct += (preds == yb).sum().item()
        total += yb.numel()

    return float(np.mean(losses)), (correct / total if total else 0.0)

In [16]:
def load_data(data_path, normalize: bool = True):
    data = np.load(data_path)
    X, y = data["X_raw"], data["y"]

    if X.ndim != 5 or y.ndim != 3:
        raise ValueError("X must be 5D and y 3D")

    n_sub, n_sess, n_chan, n_trials, n_samples = X.shape

    print(f"X original: {X.shape}")
    print(f"y original: {y.shape}")

    if normalize:
        X_mean = X.mean(axis=-1, keepdims=True)
        X_std = X.std(axis=-1, keepdims=True) + 1e-6
        X = (X - X_mean) / X_std
        
    X_transposed = X.transpose(0, 1, 3, 2, 4)  # (sub, sess, trial, chan, time)
    X_new = X_transposed.reshape(n_sub, n_sess * n_trials, n_chan, n_samples)
    y_new = y.reshape(n_sub, n_sess * n_trials)

    if X_new.shape[1] != y_new.shape[1]:
        raise RuntimeError(f"Mismatch: X_new {X_new.shape}, y_new {y_new.shape}")

    unique_labels = np.unique(y_new)
    if not set(unique_labels).issubset({0, 1}):
        raise RuntimeError(f"Non binary labels found: {unique_labels}")

    print(f"X final: {X_new.shape}")
    print(f"y final: {y_new.shape}")

    return X_new, y_new

In [17]:
import pandas as pd

def save_results_csv(results_dict, save_dir, prefix, model_args, epochs):
    rows = []
    for combo, stats in results_dict.items():
        rows.append({
            "channels": combo,
            "mean_accuracy": stats["mean"],
            "std_accuracy": stats["std"],
            "lstm_hidden": model_args["lstm_hidden"],
            "lstm_layers": model_args["lstm_layers"],
            "dropout": model_args["dropout"],
            "epochs": epochs
        })

    df = pd.DataFrame(rows)

    fname = (
        f"{prefix}_lstm_h{model_args['lstm_hidden']}"
        f"_l{model_args['lstm_layers']}"
        f"_do{model_args['dropout']}"
        f"_ep{epochs}.csv"
    )

    save_path = os.path.join(save_dir, fname)
    df.to_csv(save_path, index=False)

    print(f"Saved results to: {save_path}")
    return df


In [18]:
X, y = load_data(DATA_PATH, True)

X original: (27, 2, 5, 25, 384)
y original: (27, 2, 25)
X final: (27, 50, 5, 384)
y final: (27, 50)


# Model

In [19]:
class EEGLieFeatureExtractor(nn.Module):
    def __init__(self, n_channels=5, seq_len=384, tcn_channels=64, kernel_size=5,
                 lstm_hidden=128, lstm_layers=2, dropout=0.3):
        super().__init__()
        
        # TCN: stack of 1D convs
        self.tcn = nn.Sequential(
            nn.Conv1d(n_channels, tcn_channels, kernel_size=kernel_size, padding=kernel_size//2),
            nn.ReLU(),
            nn.LayerNorm([tcn_channels, seq_len]),
            nn.Conv1d(tcn_channels, tcn_channels, kernel_size=kernel_size, padding=kernel_size//2),
            nn.ReLU(),
            nn.LayerNorm([tcn_channels, seq_len]),
            nn.MaxPool1d(2)  # Reduce sequence length
        )
        
        # BiLSTM for long-term temporal dynamics
        self.lstm = nn.LSTM(
            input_size=tcn_channels,
            hidden_size=lstm_hidden,
            num_layers=lstm_layers,
            batch_first=True,
            bidirectional=True
        )
        
        # Head (feature vector output)
        self.classifier = nn.Linear(2 * lstm_hidden, 1)
    
    def forward(self, x):
        # TCN
        x = self.tcn(x)                     # (batch, tcn_channels, seq_len//2)
        x = x.transpose(1, 2)               # (batch, seq_len//2, tcn_channels) for LSTM
        
        # LSTM
        lstm_out, _ = self.lstm(x)          # (batch, seq_len//2, 2*lstm_hidden)
        
        return self.classifier(lstm_out[:, -1, :])  # (batch, 1)

# Cross subject LOSO

In [20]:
def cross_subject_loso(X, y, channels, model_args, epochs):
    n_subjects = X.shape[0]
    channel_indices = list(range(len(channels)))
    results = {}
    
    # Generate all non-empty combinations of channels
    all_combos = []
    for r in range(1, len(channels) + 1):
        all_combos.extend(list(itertools.combinations(channel_indices, r)))
    
    for combo in all_combos:
        combo_names = [channels[i] for i in combo]
        combo_key = ", ".join(combo_names)
        print(f"\n===== Analyzing Combo: {combo_key} =====")
        subject_accuracies = []
        
        for test_sub in range(n_subjects):
            train_subs = [i for i in range(n_subjects) if i != test_sub]
            
            # Prepare train and test tensors
            X_train = torch.FloatTensor(X[train_subs][:, :, combo, :]).reshape(-1, len(combo), 384).to(device)
            y_train = torch.FloatTensor(y[train_subs]).reshape(-1, 1).to(device)
            
            X_test = torch.FloatTensor(X[test_sub:test_sub+1][:, :, combo, :]).reshape(-1, len(combo), 384).to(device)
            y_test = y[test_sub].flatten()
            
            # Instantiate model
            model_args['n_channels'] = len(combo)
            model = EEGLieFeatureExtractor(**model_args).to(device)
            optimizer = optim.Adam(model.parameters(), lr)
            criterion = nn.BCEWithLogitsLoss()  # use logits later if needed
            
            # Training loop
            
            train_ds = TensorDataset(X_train, y_train)
            train_loader = DataLoader(train_ds, batch_size=64, shuffle=True)

            X_test_t = X_test.to(device)
            y_test_t = torch.FloatTensor(y_test).view(-1, 1).to(device)
            test_ds = TensorDataset(X_test_t, y_test_t)
            test_loader = DataLoader(test_ds, batch_size=64, shuffle=False)

            best_val = float("inf")
            best_state = None

            # Training + validation (LOSO: test subject = validation)
            for epoch in range(epochs):
                model.train()
                for xb, yb in train_loader:
                    optimizer.zero_grad()
                    logits = model(xb)
                    loss = criterion(logits, yb)
                    loss.backward()
                    optimizer.step()

                val_loss, val_acc = eval_on_loader_bce(
                    model, test_loader, device
                )

                if val_loss < best_val:
                    best_val = val_loss
                    best_state = {
                        k: v.detach().cpu().clone()
                        for k, v in model.state_dict().items()
                    }

            # Load best model
            model.load_state_dict(best_state)

            # Final evaluation (accuracy of best model)
            _, best_acc = eval_on_loader_bce(
                model, test_loader, device
            )

            subject_accuracies.append(best_acc)

        
        # Compute mean and std accuracy
        mean_acc = np.mean(subject_accuracies)
        std_acc = np.std(subject_accuracies)
        results[combo_key] = {'mean': mean_acc, 'std': std_acc}
        print(f"Combo Result -> Mean: {mean_acc:.4f} | STD: {std_acc:.4f}")
    
    return results


# Within subjects LOOCV

In [21]:
def within_subject_loocv(X, y, channels, model_args, epochs):
    n_subjects = X.shape[0]
    channel_indices = list(range(len(channels)))
    results = {}

    
    all_combos = []
    for r in range(1, len(channels) + 1):
        all_combos.extend(list(itertools.combinations(channel_indices, r)))

    for combo in all_combos:
        combo_names = [channels[i] for i in combo]
        combo_key = ", ".join(combo_names)
        print(f"\n===== Within-Subject | Combo: {combo_key} =====")

        subject_accuracies = []

        for subj in range(n_subjects):
            X_subj = X[subj][:, combo, :]      # (trials, channels, seq)
            y_subj = y[subj]                   # (trials,)

            n_trials = X_subj.shape[0]

            for test_idx in range(n_trials):
                train_idx = [i for i in range(n_trials) if i != test_idx]

                X_train = torch.FloatTensor(X_subj[train_idx]).to(device)
                y_train = torch.FloatTensor(y_subj[train_idx]).view(-1, 1).to(device)

                X_test = torch.FloatTensor(X_subj[test_idx:test_idx+1]).to(device)
                y_test = torch.FloatTensor([y_subj[test_idx]]).view(-1, 1).to(device)

                train_ds = TensorDataset(X_train, y_train)
                train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)

                test_ds = TensorDataset(X_test, y_test)
                test_loader = DataLoader(test_ds, batch_size=1, shuffle=False)

                model_args["n_channels"] = len(combo)
                model = EEGLieFeatureExtractor(**model_args).to(device)
                optimizer = optim.Adam(model.parameters(), lr)
                criterion = nn.BCEWithLogitsLoss()

                best_val = float("inf")
                best_state = None

                for epoch in range(epochs):
                    model.train()
                    for xb, yb in train_loader:
                        optimizer.zero_grad()
                        logits = model(xb)
                        loss = criterion(logits, yb)
                        loss.backward()
                        optimizer.step()

                    val_loss, _ = eval_on_loader_bce(model, test_loader, device)
                    if val_loss < best_val:
                        best_val = val_loss
                        best_state = {
                            k: v.detach().cpu().clone()
                            for k, v in model.state_dict().items()
                        }

                model.load_state_dict(best_state)
                _, acc = eval_on_loader_bce(model, test_loader, device)
                subject_accuracies.append(acc)

        mean_acc = np.mean(subject_accuracies)
        std_acc = np.std(subject_accuracies)
        results[combo_key] = {"mean": mean_acc, "std": std_acc}

        print(f"Within-Subject Result -> Mean: {mean_acc:.4f} | STD: {std_acc:.4f}")

    return results


In [22]:
'''
model_args = {
    "seq_len": 384,
    "tcn_channels": 64,
    "kernel_size": 5,
    "lstm_hidden": 128,
    "lstm_layers": 2,
    "dropout": 0.3
}

# X shape: (n_subjects, n_trials, n_channels, seq_len)
# y shape: (n_subjects, n_trials)

# Within-subject LOOCV
within_results = within_subject_loocv(X, y, CHANNELS, model_args, epochs=20)

# Cross-subject LOSO
cross_results = cross_subject_loso(X, y, CHANNELS, model_args, epochs=20)
'''

'\nmodel_args = {\n    "seq_len": 384,\n    "tcn_channels": 64,\n    "kernel_size": 5,\n    "lstm_hidden": 128,\n    "lstm_layers": 2,\n    "dropout": 0.3\n}\n\n# X shape: (n_subjects, n_trials, n_channels, seq_len)\n# y shape: (n_subjects, n_trials)\n\n# Within-subject LOOCV\nwithin_results = within_subject_loocv(X, y, CHANNELS, model_args, epochs=20)\n\n# Cross-subject LOSO\ncross_results = cross_subject_loso(X, y, CHANNELS, model_args, epochs=20)\n'

In [23]:


base_model_args = {
    
    "seq_len": 384,          
    "tcn_channels": 64,      
    "kernel_size": 5,        

    
    "lstm_hidden": None,     
    "lstm_layers": None,     
    "dropout": None          
}

param_grid = [
    {"lstm_hidden": 64,  "lstm_layers": 1, "dropout": 0.2, "lr": 1e-3},
    {"lstm_hidden": 128, "lstm_layers": 1, "dropout": 0.3, "lr": 1e-3},
    {"lstm_hidden": 128, "lstm_layers": 2, "dropout": 0.3, "lr": 5e-4},
]


In [ ]:
# ==============================
# FINAL EXPERIMENTS + SAVE
# ==============================

epochs = 10   
for cfg in param_grid:

    print("\n########################################")
    print(f"### RUNNING CONFIG: {cfg}")
    print("########################################")

    # ---- costruisci i veri model_args ----
    model_args = base_model_args.copy()
    model_args.update({
        "lstm_hidden": cfg["lstm_hidden"],
        "lstm_layers": cfg["lstm_layers"],
        "dropout": cfg["dropout"],
    })

    lr = cfg["lr"]

    print("FINAL MODEL ARGS:", model_args)
    print("Learning rate:", lr)
print("\n### RUNNING WITHIN-SUBJECT LOOCV ###")


within_results = within_subject_loocv(
    X,
    y,
    CHANNELS,
    model_args,
    epochs=epochs
)

within_df = save_results_csv(
    within_results,
    save_dir=WITHIN_DIR,
    prefix="within",
    model_args=model_args,
    epochs=epochs
)


print("\n### RUNNING CROSS-SUBJECT LOSO ###")
cross_results = cross_subject_loso(
    X,
    y,
    CHANNELS,
    model_args,
    epochs=epochs
)

cross_df = save_results_csv(
    cross_results,
    save_dir=CROSS_DIR,
    prefix="cross",
    model_args=model_args,
    epochs=epochs
)



########################################
### RUNNING CONFIG: {'lstm_hidden': 64, 'lstm_layers': 1, 'dropout': 0.2, 'lr': 0.001}
########################################
FINAL MODEL ARGS: {'seq_len': 384, 'tcn_channels': 64, 'kernel_size': 5, 'lstm_hidden': 64, 'lstm_layers': 1, 'dropout': 0.2}
Learning rate: 0.001

########################################
### RUNNING CONFIG: {'lstm_hidden': 128, 'lstm_layers': 1, 'dropout': 0.3, 'lr': 0.001}
########################################
FINAL MODEL ARGS: {'seq_len': 384, 'tcn_channels': 64, 'kernel_size': 5, 'lstm_hidden': 128, 'lstm_layers': 1, 'dropout': 0.3}
Learning rate: 0.001

########################################
### RUNNING CONFIG: {'lstm_hidden': 128, 'lstm_layers': 2, 'dropout': 0.3, 'lr': 0.0005}
########################################
FINAL MODEL ARGS: {'seq_len': 384, 'tcn_channels': 64, 'kernel_size': 5, 'lstm_hidden': 128, 'lstm_layers': 2, 'dropout': 0.3}
Learning rate: 0.0005

### RUNNING WITHIN-SUBJECT LOOCV ###

====